In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/obesity-levels/ObesityDataSet_raw_and_data_sinthetic.csv


In [2]:
import pandas as pd

# Replace with your uploaded filename
df = pd.read_csv("/kaggle/input/obesity-levels/ObesityDataSet_raw_and_data_sinthetic.csv")

# Display first few rows
df.head()


,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,21.0,Female,1.62,64.0,no,no,2.0,3.0,no,no,2.0,yes,0.0,1.0,Sometimes,Public_Transportation,Normal_Weight
1,21.0,Female,1.52,56.0,Sometimes,no,3.0,3.0,yes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,23.0,Male,1.80,77.0,Frequently,no,2.0,3.0,no,no,2.0,yes,2.0,1.0,Sometimes,Public_Transportation,Normal_Weight
3,27.0,Male,1.80,87.0,Frequently,no,3.0,3.0,no,no,2.0,no,2.0,0.0,Sometimes,Walking,Overweight_Level_I
4,22.0,Male,1.78,89.8,Sometimes,no,2.0,1.0,no,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [4]:
constant_features = [col for col in x_train.columns if x_train[col].nunique() == 1]

print(constant_features)

[]


In [5]:
quasi_constant_features = []

# Set the threshold (98%)
threshold = 0.98

for col in x_train.columns:
    freq = x_train[col].value_counts(normalize=True).values[0]
    if freq >= threshold:
        quasi_constant_features.append(col)

print( quasi_constant_features)



[]


In [6]:
import numpy as np

# Drop constant columns
non_constant_cols = [col for col in x_train.columns if x_train[col].nunique() > 1]
x_train_non_constant = x_train[non_constant_cols]

# Correlation matrix
corr_matrix = x_train_non_constant.corr(numeric_only=True).abs()

# Upper triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).fillna(0)

# Find highly correlated features
correlated_features = [col for col in upper.columns if any(upper[col] > 0.90)]

print("Highly Correlated Features:", correlated_features)


Highly Correlated Features: []


In [7]:
from sklearn.feature_selection import mutual_info_classif

# For MI, encode categorical variables
x_train_encoded = pd.get_dummies(x_train)
x_test_encoded = pd.get_dummies(x_test)

# Ensure both train and test have the same columns
x_test_encoded = x_test_encoded.reindex(columns=x_train_encoded.columns, fill_value=0)

# Calculate MI scores
mi_scores = mutual_info_classif(x_train_encoded, y_train)

# Sort and print
mi_series = pd.Series(mi_scores, index=x_train_encoded.columns).sort_values(ascending=False)

print("Top 10 Features by Mutual Information:")
print(mi_series.head(10))


Top 10 Features by Mutual Information:
Weight           1.235674
Age              0.569793
Height           0.444377
FCVC             0.373120
CH2O             0.299196
FAF              0.295591
TUE              0.280131
NCP              0.245538
Gender_Male      0.204690
Gender_Female    0.202372
dtype: float64
